In [1]:
import pickle
from datetime import date,time,datetime,timedelta
from freezegun import freeze_time

#freezer = freeze_time("2022-05-21 15:45:01")
#freezer.start()
#assert datetime.now() == datetime(2022, 5, 21, 15, 45, 1)


In [2]:
class habit:
    '''The class habit'''
    def __init__(self,name,periodicity):
        '''function used to instatiate the class'''
        self.name = name
        self.created = datetime.now()
        self.periodicity = periodicity
        self.streak = 0
        self.current_streak = 0
        self.last_action = datetime.now()
        self.next_action = datetime.now()
        self.last_checkoff = 0
        self.check_already_checkoff = 0
    def set_action_time(self):
        '''function used to set the first next action time of a habit using information
        provided by the user 0 for daily 1 for weekly.'''
        if self.periodicity == 0:
            self.next_action = datetime.now()+ timedelta(1)
        elif self.periodicity == 1:
            self.next_action = datetime.now()+ timedelta(7)
            
    def checkoff_performed_action(self):
        ''' function of habit class used to check-off the habit intented
        by the user and check if the habit is already check-off or not
        this way deciding if the checking counts or not to the streak'''
        
        
        self.last_action = datetime.now()
        if self.periodicity == 0 and self.check_already_checkoff == 0:
         #for the daily habits   
            self.check_already_checkoff += 1
            self.current_streak += 1
            self.last_checkoff = datetime.now()
            if self.current_streak > self.streak:
                self.streak = self.current_streak
                
        
        elif self.periodicity == 1 and self.check_already_checkoff == 0:
             #for the weekly habits       
            self.check_already_checkoff += 1
            self.current_streak += 1
            self.last_checkoff = datetime.now()
            if self.current_streak > self.streak:
                self.streak = self.current_streak
                
        
    def check_time_left(self):
        '''To check if any streaks were broken and alert the user
        and/or to reset the already checkoff counter and set the next action
        limit'''
        if self.periodicity == 0:
            if datetime.now() > self.next_action and self.check_already_checkoff == 1:
                self.next_action = self.next_action + timedelta(1)
                self.check_already_checkoff = 0
            if datetime.now() > self.next_action and self.check_already_checkoff == 0:
                print('You broke the streak for',self.name )
                self.current_streak = 0
                self.next_action = self.next_action + timedelta(1)
        elif self.periodicity == 1:  
            if datetime.now() > self.next_action and self.check_already_checkoff == 1:
                self.next_action = self.next_action + timedelta(7)
                self.check_already_checkoff = 0
            if datetime.now() > self.next_action and self.check_already_checkoff == 0:
                print('You broke the streak for',self.name )
                self.current_streak = 0    
                self.next_action = self.next_action + timedelta(7)

In [3]:
def welcome_message():
    '''This is used in the main loop of the program
    for giving 6 options to the user which can decide what does he/she intend'''
    
    
    option = input('''\nWhat would you like to do?
    Enter 0 if you would like to check-off an habit 
    Enter 1 if you would like to add a new habit
    Enter 2 if you would like to see your active habits
    Enter 3 if you would like to check your analytics 
    Enter 4 if you would like to delete a habit
    Enter 5 if you want the program to quit\n''')
    
    try: #checking if the user input is valid
        
        option = int(option)
        
    except ValueError:
        
        print('\nYou must enter a valid option\n')
        
        welcome_message()
        
    
    if option > 5:
        print('\nYou must enter a valid option\n')
        welcome_message()
    if option <0 :
        print('\nYou must enter a valid option\n')
        welcome_message()
    
    if option == 0: # Allows the user to checkoff an habit and checks if a habit has been created before
        if len(habits) > 0:
            all_the_habits = [h.name for h in habits]
            print(all_the_habits)
            check_off()
        elif len(habits) == 0:
            print('\nYou need to create a habit first!\n')
            welcome_message()
        
        
        
    elif option == 1:  # Creates the instance of the class and append the object to the habits list              
            habit_input = choose_habit()
            habit_periodicity = choose_periodicity()
            habit_instance = habit(habit_input,habit_periodicity)
            habit_instance.set_action_time()
            habits.append(habit_instance)
    elif option == 2:
            
        give_als_info() # Shows the user info about current habits
    elif option == 3:
            
        choose_analytic() # provides the user with a menu for analytics
    
    elif option == 4: # checks if a habit exists to stop errors and allows the user to delete a habit
        if len(habits) >0:
            delete_habit()
        elif len(habits) == 0:
            print('\nYou need to create a habit first!\n')
            welcome_message()
    elif option == 5: # stops the main loop of the program and clears and saves objects to the pickle file 
        global simple_counter
        simple_counter += 1
        open('habitsfile.pickle','w').close()
        for iter_2 in habits:
            save_object(iter_2,'habitsfile.pickle')

def choose_habit ():  
    
    '''To let the user choose a name for a habit'''
    
    habit_name = input('''\nInsert the name of the habit you wish to add\n''')
    return habit_name

def choose_periodicity(): 
    
    '''to allow the user to choose the periodicity'''
    
    habit_periodicity = input('''\nInsert the periodicity you desire, 
    enter 1 for weekly or 0 for daily\n''')
    try: 
        
        habit_periodicity = int(habit_periodicity)
        
    except ValueError:
        
        print('\nYou must enter a valid option\n')
        
        choose_periodicity()
    if habit_periodicity < 0 or habit_periodicity >1 :
        
        print('\nYou must enter a valid option\n')
        choose_periodicity()
    elif habit_periodicity <= 1 and habit_periodicity >=0:
        return habit_periodicity
        

def save_object(obj, filename): 
    
    '''Appends objects to the file'''
    
    with open(filename, 'ab') as outp:  
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
def load_object():
    
    '''Loads objects from the pickle file to the habits list which at the
    start of every session is empty'''
    
    open('habitsfile.pickle','a+')
    with (open("habitsfile.pickle", "rb")) as openfile:
        while True:
            try:
                habits.append(pickle.load(openfile))
            except EOFError:
                break
def give_als_info():
    
    '''Gives information about the habits for option number 2 of the user'''
    
    for info in habits:
        if info.check_already_checkoff == 0:
            information = 'not yet checked-off'
        elif info.check_already_checkoff == 1:
            information = 'already checked-off'
        print('\n',info.name,'created on:',info.created.ctime(),'last action on', info.last_action.ctime(),'next action until',info.next_action.ctime(),information,'current streak :',info.current_streak)
def choose_analytic():
    
    '''Allows the user to choose an analytic from the menu
    the details of each option are described in the input'''
    
    option_analytic = input('''Enter 0 for a list of daily habits
Enter 1 for a list of weekly habits
Enter 2 for the longest streak amongst all habits
Enter 3 for the longest streak for a specific habit
Enter 4 to go back\n''')
    try:
        
        option_analytic = int(option_analytic)
        
    except ValueError:
        
        print('\nYou must enter a valid option\n')
        
        choose_analytic()
    if option_analytic < 0 or option_analytic >4 :
        
        print('\nYou must enter a valid option\n')
        choose_analytic()
    
    elif option_analytic == 0:
        list_show = []
        for iter_1 in habits:
            if iter_1.periodicity == 0:
                list_show.append(iter_1.name)
        for iter_1 in list_show:
            print('\n',iter_1)
                
    elif option_analytic == 1:
        list_show = []
        for iter_1 in habits:
            if iter_1.periodicity == 1:
                list_show.append(iter_1.name)
        for iter_1 in list_show:
            print('\n',iter_1)
            
    elif option_analytic == 2:
        counter_2 = -1
        biggest_streak = None
        for iter_1 in habits:
            counter_1 = iter_1.streak
            if counter_1 > counter_2:
                biggest_streak = iter_1.name,iter_1.streak
                counter_2 = counter_1
            elif counter_1 == counter_2:
                biggest_streak = biggest_streak + (iter_1.name,iter_1.streak)
                
        print('\n',biggest_streak)  
        
    elif option_analytic == 3:
        choosen_habit = input('''Insert the name of the habit you which to see the longest streak\n''')
        list_of_habits = set([habit_ele_4.name for habit_ele_4 in habits])
        list_of_habits = list(list_of_habits)
        if list_of_habits.count(choosen_habit)>0:
            
            for habit_ele_4 in habits:
                if habit_ele_4.name == choosen_habit:
                    
                    print('\n',habit_ele_4.name,habit_ele_4.streak)
    elif option_analytic == 4:
        welcome_message()
def delete_habit():
    '''deletes habits'''
    habit_to_delete = input('\nInsert the name of the habit you wish to delete\n')
    list_of_habits_2 = [habit_ele.name for habit_ele in habits]
    if list_of_habits_2.count(habit_to_delete)>0:
        
        for habit_deletion in habits:
            
            if habit_deletion.name == habit_to_delete:
                habits.remove(habit_deletion)
                
                welcome_message()
    elif list_of_habits_2.count(habit_to_delete) == 0:    
        print('\nHabit not in the list! Insert a valid habit\n')
        delete_habit()

def check_off():
    '''Allows the user to checkoff habits'''
    habit_to_check_off = input('\nInsert the name of the habit you wish to check-off\n')
    list_of_habits_3 = set([habit_ele_1.name for habit_ele_1 in habits])
    list_of_habits_3 = list(list_of_habits_3)
    
    if list_of_habits_3.count(habit_to_check_off)>0:
        
        for habit_checking in habits:
            
            
            if habit_checking.name == habit_to_check_off:
                habit_checking.checkoff_performed_action()
                welcome_message()
    elif list_of_habits_3.count(habit_to_check_off) == 0:    
        print('\nHabit not in the list! Insert a valid habit\n')
        check_off()

                

In [ ]:
habits = list() 

# List to store all the instances of the habit class


simple_counter = 0 # a counter that if is set to 1 stops the program

load_object() # at the start of every session the objects in the pickle file are loaded to an empty habits list

while simple_counter ==0 :
    if __name__ == '__main__': # Main loop
        
        
        
        for objects in habits: # Checks if the streak has been broken for each habit
            objects.check_time_left()
        
        
            
        welcome_message() # shows the welcome message
        
            